In [15]:
#创建：2016/01/26
#文件：BarCodeIdentification.py
#作者：moverzp
#功能：识别条形码
import sys
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import os

DECODING_TABLE = {
    '0001101': 0, '0100111': 0, '1110010': 0,
    '0011001': 1, '0110011': 1, '1100110': 1,
    '0010011': 2, '0011011': 2, '1101100': 2,
    '0111101': 3, '0100001': 3, '1000010': 3,
    '0100011': 4, '0011101': 4, '1011100': 4,
    '0110001': 5, '0111001': 5, '1001110': 5,
    '0101111': 6, '0000101': 6, '1010000': 6,
    '0111011': 7, '0010001': 7, '1000100': 7,
    '0110111': 8, '0001001': 8, '1001000': 8,
    '0001011': 9, '0010111': 9, '1110100': 9,    
    }

EDGE_TABLE = {
    2:{2:6,3:0,4:4,5:3},
    3:{2:9,3:'33',4:'34',5:5},
    4:{2:9,3:'43',4:'44',5:5},
    5:{2:6,3:0,4:4,5:3},
    }

FIRST_TABLE = {
    2:{2:'O',3:'E',4:'O',5:'E'},
    3:{2:'E',3:'O',4:'E',5:'O'},
    4:{2:'O',3:'E',4:'O',5:'E'},
    5:{2:'E',3:'O',4:'E',5:'O'},
    }

FIRST_DETECT_TABLE = {
    'OOOOOO':0, 'OOEOEE':1, 'OOEEOE':2, 'OOEEEO':3, 'OEOOEE':4, 
    'OEEOOE':5, 'OEEEOO':6, 'OEOEOE':7, 'OEOEEO':8, 'OEEOEO':9
    }


INDEX_IN_WIDTH = (0, 4, 8, 12, 16, 20, 24, 33, 37, 41, 45, 49, 53)
def get_bar_space_width(img, h):
    #row = img.shape[0] *1/2
#     row = int(img.shape[0] *1/2)
    row = h
    currentPix = -1
    lastPix = -1
    pos = 0
    width = []
    for i in range(img.shape[1]):#遍历一整行
        currentPix = img[row][i]
        if currentPix != lastPix:
            if lastPix == -1:
                lastPix = currentPix
                pos = i
            else:
                width.append( i - pos )
                pos = i
                lastPix = currentPix
    return width
    
def divide(t, l):
    if float(t) / l < 0.357:
        return 2
    elif float(t) / l < 0.500:
        return 3
    elif float(t) / l < 0.643:
        return 4
    else:
        return 5
    
def cal_similar_edge(data):
    similarEdge = []
    #先判断起始符
    limit = float(data[1] + data[2] + data[3] ) / 3 * 1.5
    if data[1] >= limit or data[2] >= limit or data[3] >= limit:
        return -1#宽度提取失败
    index = 4
    while index < 54:
        #跳过分隔符区间
        if index==28 or index==29 or index==30 or index==31 or index==32:
            index +=1
            continue
        #字符检测
        T1 = data[index] + data[index+1]
        T2 = data[index+1] + data[index+2]
        L = data[index] + data[index+1] + data[index+2] + data[index+3]
        similarEdge.append( divide(T1, L) )
        similarEdge.append( divide(T2, L) )
        index += 4
 
    return similarEdge
    
def decode_similar_edge(edge):
   
    #第一个字符一定是6，中国区
    first = [];
    for i in range(0, 12, 2):
        first.append(FIRST_TABLE[edge[i]][edge[i+1]])
    str = "".join(first)
    barCode = [FIRST_DETECT_TABLE[str]]
    
    
    for i in range (0, 24, 2):#每个字符两个相似边，共12个字符
        barCode.append( EDGE_TABLE[edge[i]][edge[i+1]] )
    return barCode
    
def decode_sharp(barCode, barSpaceWidth):
    for i in range(0, 13):
        if barCode[i] == '44':
            index = INDEX_IN_WIDTH[i]
            c3 = barSpaceWidth[index+2]
            c4 = barSpaceWidth[index+3]
            if c3 > c4:
                barCode[i] = 1
            else:
                barCode[i] = 7            
        elif barCode[i] == '33':
            index = INDEX_IN_WIDTH[i]
            c1 = barSpaceWidth[index]
            c2 = barSpaceWidth[index+1]
            if c1 > c2:
                barCode[i] = 2
            else:
                barCode[i] = 8
        elif barCode[i] == '34':
            index = INDEX_IN_WIDTH[i]
            c1 = barSpaceWidth[index]
            c2 = barSpaceWidth[index+1]
            if c1 > c2:
                barCode[i] = 7
            else:
                barCode[i] = 1            
        elif barCode[i] == '43':
            index = INDEX_IN_WIDTH[i]
            c2 = barSpaceWidth[index+1]
            c3 = barSpaceWidth[index+2]
            if c2 > c3:
                barCode[i] = 2
            else:
                barCode[i] = 8

def check_bar_code(barCode):
    evens = barCode[11]+barCode[9]+barCode[7]+barCode[5]+barCode[3]+barCode[1]
    odds = barCode[10]+barCode[8]+barCode[6]+barCode[4]+barCode[2]+barCode[0]
    sum = evens * 3 + odds
    if barCode[12] == (10 - sum % 10) % 10:
        return True
    else:
        return False
#载入图像
#img = cv2.imread('image.png')
# img = cv2.imread('barcode.jpg')






In [11]:
#载入图像
#img = cv2.imread('image.png')
# img = cv2.imread('barcode.jpg')
path = "D:/ean13/BarcodeImage"
img = cv2.imread(path + '/' +'6.png')
grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#转换成单通道图像
grayImg = cv2.medianBlur(grayImg, 3)#中值滤波
ret, grayImg = cv2.threshold(grayImg, 200, 255, cv2.THRESH_BINARY)#二值化

In [12]:
print(img.shape[1])
print(int(img.shape[0]*1/2))

567
142


In [13]:
#提取条空宽度
barSpaceWidth = get_bar_space_width(grayImg)
print ('bar & space\'s numbers:%d' %(len(barSpaceWidth)))#只有60是正确的
print (barSpaceWidth)


bar & space's numbers:60
[56, 5, 5, 5, 5, 15, 5, 10, 10, 10, 10, 5, 20, 5, 5, 5, 5, 5, 5, 20, 5, 5, 20, 6, 10, 15, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 20, 5, 10, 5, 15, 5, 20, 5, 5, 5, 10, 5, 15, 10, 5, 10, 10, 11, 10, 10, 5, 5, 5, 5]


In [14]:
#计算相似边数值
similarEdge = cal_similar_edge(barSpaceWidth)
if similarEdge == -1:
    print ('barSpaceWidth error!')
    sys.exit()
print ('similarEdge\'s numbers:%d'%(len(similarEdge)))
print (similarEdge)
#相似边译码
barCode = decode_similar_edge(similarEdge)
#针对‘#’译码
decode_sharp(barCode, barSpaceWidth)
#校验
valid = check_bar_code(barCode)
valid = 1
print ('barcode:\n', barCode if valid else 'Check barcode error!')

height = img.shape[0]
width = img.shape[1]
#cv2.line(grayImg, (0, height/2), (width, height/2),(0, 255, 0), 2)#画出扫描的行
cv2.line(grayImg, (0, int(height/2)), (width, int(height/2)),(0, 255, 0), 2)#画出扫描的行
#显示图像
# cv2.imshow("origin", img)
# cv2.imshow("result", grayImg)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
#cv2.waitKey(1)

# key = cv2.waitKey(0)
# if key == 27:
#     cv2.destroyAllWindows()


similarEdge's numbers:24
[4, 4, 4, 4, 5, 2, 2, 2, 2, 5, 5, 4, 2, 2, 3, 3, 5, 5, 3, 3, 3, 3, 4, 4]
barcode:
 [1, 7, 1, 6, 6, 3, 4, 6, 8, 3, 8, 2, 1]


In [23]:
def decode(img):
#     img = cv2.imread(path)
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#转换成单通道图像
    grayImg = cv2.medianBlur(grayImg, 3)#中值滤波
    ret, grayImg = cv2.threshold(grayImg, 200, 255, cv2.THRESH_BINARY)#二值化
    
    height = img.shape[0]
    for i in range(height):
        #提取条空宽度
        barSpaceWidth = get_bar_space_width(grayImg, i)
        if len(barSpaceWidth) >=60:
            similarEdge = cal_similar_edge(barSpaceWidth)
    #         if similarEdge == -1:
    #             print ('barSpaceWidth error!')
    #             sys.exit()
    #         print ('similarEdge\'s numbers:%d'%(len(similarEdge)))
    #         print (similarEdge)
            #相似边译码
            barCode = decode_similar_edge(similarEdge)
            #针对‘#’译码
            decode_sharp(barCode, barSpaceWidth)
            #校验
            valid = check_bar_code(barCode)
            if valid == 1:
                break
        
    return barCode

In [24]:
path = "D:/ean13/BarcodeImage"
imgpath = path + '/' +'6.png'
image = cv2.imread(imgpath)
barcode = decode(image)

print(barcode)

[1, 7, 1, 6, 6, 3, 4, 6, 8, 3, 8, 2, 1]


In [32]:
path = "D:/ean13/BarcodeImage"
filelist = os.listdir(path)
sum = 0
success = 0
i = 0
excelBarcode = pd.read_excel("D:/ean13/ean13.xlsx", header = None)


for i in range(1000):
    imagePath = path + "/" + str(i+1) + ".png"
    sum = sum + 1
    # barCode = barcode_detection(imagePath)
    image = cv2.imread(imagePath)
    barCode = decode(image)
    barCode_string = ""
    for item in barCode:
        barCode_string = barCode_string + str(item)

    if barCode_string == str(excelBarcode.iloc[i, 0]):
        success = success + 1
    print(imagePath)
    print(i)
    print(barCode_string)
    print(str(excelBarcode.iloc[i,0]))
   

print(sum)
print(success)
print(success / sum)

D:/ean13/BarcodeImage/1.png
0
2585911494387
2585911494387
D:/ean13/BarcodeImage/2.png
1
3292073773203
3292073773203
D:/ean13/BarcodeImage/3.png
2
6204547942883
6204547942883
D:/ean13/BarcodeImage/4.png
3
3724002996563
3724002996563
D:/ean13/BarcodeImage/5.png
4
4768469352646
4768469352646
D:/ean13/BarcodeImage/6.png
5
1716634683821
1716634683821
D:/ean13/BarcodeImage/7.png
6
6658977068031
6658977068031
D:/ean13/BarcodeImage/8.png
7
3157672007671
3157672007671
D:/ean13/BarcodeImage/9.png
8
3473415519349
3473415519349
D:/ean13/BarcodeImage/10.png
9
3683511942891
3683511942891
D:/ean13/BarcodeImage/11.png
10
6012194211784
6012194211784
D:/ean13/BarcodeImage/12.png
11
5453203307059
5453203307059
D:/ean13/BarcodeImage/13.png
12
9466612526658
9466612526658
D:/ean13/BarcodeImage/14.png
13
7795145993111
7795145993111
D:/ean13/BarcodeImage/15.png
14
4314836748378
4314836748378
D:/ean13/BarcodeImage/16.png
15
9281843452701
9281843452701
D:/ean13/BarcodeImage/17.png
16
0286891449726
286891449726


D:/ean13/BarcodeImage/141.png
140
9141440264094
9141440264094
D:/ean13/BarcodeImage/142.png
141
1858927680466
1858927680466
D:/ean13/BarcodeImage/143.png
142
8904665602697
8904665602697
D:/ean13/BarcodeImage/144.png
143
4783024230535
4783024230535
D:/ean13/BarcodeImage/145.png
144
7764189653374
7764189653374
D:/ean13/BarcodeImage/146.png
145
1493278281797
1493278281797
D:/ean13/BarcodeImage/147.png
146
1547230513973
1547230513973
D:/ean13/BarcodeImage/148.png
147
1976016136387
1976016136387
D:/ean13/BarcodeImage/149.png
148
1240014094148
1240014094148
D:/ean13/BarcodeImage/150.png
149
5603319497841
5603319497841
D:/ean13/BarcodeImage/151.png
150
2473162327496
2473162327496
D:/ean13/BarcodeImage/152.png
151
4946018886921
4946018886921
D:/ean13/BarcodeImage/153.png
152
0881350992777
881350992777
D:/ean13/BarcodeImage/154.png
153
2888451858863
2888451858863
D:/ean13/BarcodeImage/155.png
154
1187876114051
1187876114051
D:/ean13/BarcodeImage/156.png
155
9443340983465
9443340983465
D:/ean13/

D:/ean13/BarcodeImage/282.png
281
1893653949197
1893653949197
D:/ean13/BarcodeImage/283.png
282
2415772318402
2415772318402
D:/ean13/BarcodeImage/284.png
283
7250174557371
7250174557371
D:/ean13/BarcodeImage/285.png
284
6648555868207
6648555868207
D:/ean13/BarcodeImage/286.png
285
3753255016193
3753255016193
D:/ean13/BarcodeImage/287.png
286
9840937037097
9840937037097
D:/ean13/BarcodeImage/288.png
287
5001037616498
5001037616498
D:/ean13/BarcodeImage/289.png
288
9593504172003
9593504172003
D:/ean13/BarcodeImage/290.png
289
2841693728448
2841693728448
D:/ean13/BarcodeImage/291.png
290
2749850943423
2749850943423
D:/ean13/BarcodeImage/292.png
291
2411400393377
2411400393377
D:/ean13/BarcodeImage/293.png
292
2616156359640
2616156359640
D:/ean13/BarcodeImage/294.png
293
9080561820744
9080561820744
D:/ean13/BarcodeImage/295.png
294
6979054969615
6979054969615
D:/ean13/BarcodeImage/296.png
295
6227039064146
6227039064146
D:/ean13/BarcodeImage/297.png
296
0767590035835
767590035835
D:/ean13/

D:/ean13/BarcodeImage/418.png
417
7634718167897
7634718167897
D:/ean13/BarcodeImage/419.png
418
3434863084735
3434863084735
D:/ean13/BarcodeImage/420.png
419
8812332747193
8812332747193
D:/ean13/BarcodeImage/421.png
420
6886661629583
6886661629583
D:/ean13/BarcodeImage/422.png
421
7768151801596
7768151801596
D:/ean13/BarcodeImage/423.png
422
3941750860308
3941750860308
D:/ean13/BarcodeImage/424.png
423
1179903359542
1179903359542
D:/ean13/BarcodeImage/425.png
424
4237771986028
4237771986028
D:/ean13/BarcodeImage/426.png
425
5869151069312
5869151069312
D:/ean13/BarcodeImage/427.png
426
2851847509434
2851847509434
D:/ean13/BarcodeImage/428.png
427
8748344513627
8748344513627
D:/ean13/BarcodeImage/429.png
428
6377089974480
6377089974480
D:/ean13/BarcodeImage/430.png
429
5916879045347
5916879045347
D:/ean13/BarcodeImage/431.png
430
5705230869292
5705230869292
D:/ean13/BarcodeImage/432.png
431
5271554489466
5271554489466
D:/ean13/BarcodeImage/433.png
432
7325230023555
7325230023555
D:/ean13

D:/ean13/BarcodeImage/568.png
567
9915077686372
9915077686372
D:/ean13/BarcodeImage/569.png
568
5586354869982
5586354869982
D:/ean13/BarcodeImage/570.png
569
2909706693715
2909706693715
D:/ean13/BarcodeImage/571.png
570
8802018869950
8802018869950
D:/ean13/BarcodeImage/572.png
571
8440308305087
8440308305087
D:/ean13/BarcodeImage/573.png
572
1245189940481
1245189940481
D:/ean13/BarcodeImage/574.png
573
2356881078682
2356881078682
D:/ean13/BarcodeImage/575.png
574
7087630829839
7087630829839
D:/ean13/BarcodeImage/576.png
575
4918650014994
4918650014994
D:/ean13/BarcodeImage/577.png
576
3915988805400
3915988805400
D:/ean13/BarcodeImage/578.png
577
2819098267008
2819098267008
D:/ean13/BarcodeImage/579.png
578
7855989726426
7855989726426
D:/ean13/BarcodeImage/580.png
579
6795398475861
6795398475861
D:/ean13/BarcodeImage/581.png
580
8987075389587
8987075389587
D:/ean13/BarcodeImage/582.png
581
6797845935973
6797845935973
D:/ean13/BarcodeImage/583.png
582
8866927612625
8866927612625
D:/ean13

D:/ean13/BarcodeImage/716.png
715
3086817478062
3086817478062
D:/ean13/BarcodeImage/717.png
716
2798323168917
2798323168917
D:/ean13/BarcodeImage/718.png
717
9189702326777
9189702326777
D:/ean13/BarcodeImage/719.png
718
6789733183078
6789733183078
D:/ean13/BarcodeImage/720.png
719
7189355734044
7189355734044
D:/ean13/BarcodeImage/721.png
720
9760106215586
9760106215586
D:/ean13/BarcodeImage/722.png
721
7632555799295
7632555799295
D:/ean13/BarcodeImage/723.png
722
7612247257753
7612247257753
D:/ean13/BarcodeImage/724.png
723
6058336425589
6058336425589
D:/ean13/BarcodeImage/725.png
724
0861410127103
861410127103
D:/ean13/BarcodeImage/726.png
725
0720989697717
720989697717
D:/ean13/BarcodeImage/727.png
726
7434288891174
7434288891174
D:/ean13/BarcodeImage/728.png
727
7480040911260
7480040911260
D:/ean13/BarcodeImage/729.png
728
9053533997751
9053533997751
D:/ean13/BarcodeImage/730.png
729
8700583378016
8700583378016
D:/ean13/BarcodeImage/731.png
730
9975046010676
9975046010676
D:/ean13/B

D:/ean13/BarcodeImage/871.png
870
4681084551563
4681084551563
D:/ean13/BarcodeImage/872.png
871
8007693093946
8007693093946
D:/ean13/BarcodeImage/873.png
872
7670861860612
7670861860612
D:/ean13/BarcodeImage/874.png
873
7576652022125
7576652022125
D:/ean13/BarcodeImage/875.png
874
1448818842846
1448818842846
D:/ean13/BarcodeImage/876.png
875
4832788638528
4832788638528
D:/ean13/BarcodeImage/877.png
876
0672205383952
672205383952
D:/ean13/BarcodeImage/878.png
877
5918221918668
5918221918668
D:/ean13/BarcodeImage/879.png
878
4103204357102
4103204357102
D:/ean13/BarcodeImage/880.png
879
9104030175828
9104030175828
D:/ean13/BarcodeImage/881.png
880
6444639633576
6444639633576
D:/ean13/BarcodeImage/882.png
881
9906443935224
9906443935224
D:/ean13/BarcodeImage/883.png
882
5777993389843
5777993389843
D:/ean13/BarcodeImage/884.png
883
5123460005578
5123460005578
D:/ean13/BarcodeImage/885.png
884
7662328035862
7662328035862
D:/ean13/BarcodeImage/886.png
885
7740479340087
7740479340087
D:/ean13/